In [1]:
import torch
import numpy as np
from tqdm import tqdm

In [15]:
model_path = "helix01_mse.chkpt"
chkpt = torch.load(model_path)
opt = chkpt["settings"]
epoch = chkpt["epoch"]
model_state = chkpt["model"]

In [16]:
import transformer.Models
from transformer.Optim import ScheduledOptim
import torch.optim as optim
import torch.utils.data
from dataset import ProteinDataset, paired_collate_fn
from train import cal_performance

In [17]:
device = torch.device('cuda' if opt.cuda else 'cpu')
the_model = transformer.Models.Transformer(opt.max_token_seq_len,
        d_k=opt.d_k,
        d_v=opt.d_v,
        d_model=opt.d_model,
        d_inner=opt.d_inner_hid,
        n_layers=opt.n_layers,
        n_head=opt.n_head,
        dropout=opt.dropout)
the_model.load_state_dict(model_state)

In [18]:
data = torch.load(opt.data)
data_loader = torch.utils.data.DataLoader(
        ProteinDataset(
            seqs=data['test']['seq'],
            angs=data['test']['ang']),
        num_workers=2,
        batch_size=1,#opt.batch_size,
        collate_fn=paired_collate_fn,
        shuffle=True)

In [19]:
with torch.no_grad():
    for batch in tqdm(
            data_loader, mininterval=2,
            desc='  - (Evaluation) ', leave=False):

        # prepare data
        src_seq, src_pos, tgt_seq, tgt_pos = batch
        gold = tgt_seq[:]

        # forward
        pred = the_model(src_seq, src_pos, tgt_seq, tgt_pos)
        loss = cal_performance(pred, gold, device)
        print("Loss: {0:.2f}, Predshape: {1}".format(loss, pred.shape))

Loss: 0.16, Predshape: torch.Size([1, 21, 11])
Loss: 0.16, Predshape: torch.Size([1, 21, 11])


In [ ]:
np.array(pred)

In [ ]:
del struct

In [20]:
import transformer.Structure as struct
import pickle

In [22]:
coords = struct.angles2coords(pred[0], torch.device("cpu"))

In [23]:
with open("helix01_mse.pkl", "wb") as f:
    d = {"5XDJ_A": np.asarray(coords)}
    pickle.dump(d, f, 2)

In [11]:
!python coords2pdb.py helix01_pred.pkl

/home/jok120/build/anaconda3/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/jok120/build/anaconda3/lib/python2.7/site-packages/scipy/linalg/__init__.py:202: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
Traceback (most recent call last):
  File "coords2pdb.py", line 15, in <module>
    from prody import *
  File "/home/jok120/build/anaconda3/lib/python2.7/site-packages/prody/__init__.py", line 85, in <module>
    from . import proteins
  File "/home/jok120/build/anaconda3/lib/python2.7/site-packages/prody/proteins/__init__.py", line 162, in <module>
    from . import compare
  File "/home/jok120/build/anaconda3/lib/python2.7/site-packages/prody/proteins/compare.py", line 17, in <module>
    from prody.sequence import MSA
  File "/home/jok120/build/anaco

In [ ]:
len(coords)